In [1]:
import numpy as np
import pandas as pd
import torch
import random
import torchvision
from torchvision import transforms, models, datasets
import matplotlib.pyplot as plt
from torchvision.transforms import functional
from torchmetrics import Recall, Precision
from collections import defaultdict
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim
from PIL import Image

In [2]:
#Load the nutrition data (CSV file)
nutrition_df = pd.read_csv(r"D:\FRUITSANDVEGETABLESDATASET\fruitsandvegetablescropped - Copy1.csv")
# Serialize the nutrition data
nutrition_data = nutrition_df.to_dict(orient='list')

In [3]:
# Define the directory and transformations
directory = (r"D:\FRUITSANDVEGETABLESDATASET\Fruit And Vegetable Diseases Dataset - Copy")
transformer = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

data = ImageFolder(root=directory, transform=transformer)
class_names = data.classes

In [4]:
# Split the data into train, validation, and test datasets
generator1 = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(data, [0.7, 0.2, 0.1], generator=generator1)

In [5]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True) 

In [6]:
# Define VGG-16 Model
model = models.vgg16(pretrained=True)

E:\AISC LABS\ANACONDA\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\AISC LABS\ANACONDA\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# Modify the classifier to match the number of classes
num_classes = len(class_names)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)

In [8]:
# Freeze all layers except the final fully connected layer
for param in model.features.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    if param.shape[0] == num_classes:  # Only unfreeze the last layer
        param.requires_grad = True

In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.95)

In [10]:
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [11]:
# Training loop
num_epochs = 20  # Set number of epochs to 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    print(f'Validation Loss: {val_loss/len(val_loader)}, Validation Accuracy: {100 * val_correct / val_total}')

    # Test phase
    test_loss = 0.0
    test_correct = 0
    test_total = 0
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f'Test Loss: {test_loss/len(test_loader)}, Test Accuracy: {100 * test_correct / test_total}')

# Save the trained model along with the nutrition data
model_save_path = "vgg16_model.pth" 
model_state = {
    'state_dict': model.state_dict(),
    'nutrition_data': nutrition_data,
    'class_names': class_names
}

torch.save(model_state, model_save_path)
print(f'Model saved to {model_save_path}')

E:\AISC LABS\ANACONDA\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/20], Loss: 0.7657394759449963
Validation Loss: 0.38210254154429446, Validation Accuracy: 88.28951860703312
Test Loss: 0.37923633128040185, Test Accuracy: 88.25537726186411
Epoch [2/20], Loss: 0.41677801557431543
Validation Loss: 0.28747432761856073, Validation Accuracy: 91.27688630932059
Test Loss: 0.3084361200959867, Test Accuracy: 90.47456469784909
Epoch [3/20], Loss: 0.3259483407263887
Validation Loss: 0.26778603877430673, Validation Accuracy: 91.68658245134858
Test Loss: 0.2592625808851469, Test Accuracy: 91.87435984977809
Epoch [4/20], Loss: 0.27641788214955815
Validation Loss: 0.2052011837658145, Validation Accuracy: 93.47900307272107
Test Loss: 0.20417963686581378, Test Accuracy: 93.64970979856606
Epoch [5/20], Loss: 0.23142455027194608
Validation Loss: 0.21166472708202433, Validation Accuracy: 93.39364970979857
Test Loss: 0.1983204746287336, Test Accuracy: 93.58142710822807
Epoch [6/20], Loss: 0.2048854758833497
Validation Loss: 0.20203114555589094, Validation Accuracy